In [1]:
>>> import sqlalchemy
>>> sqlalchemy.__version__  

'1.3.11'

In [2]:
>>> from sqlalchemy import create_engine
>>> engine = create_engine('sqlite:///:memory:', echo=True)

In [3]:
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey
metadata = MetaData()
users = Table('users', metadata,
    Column('id', Integer, primary_key=True),
    Column('name', String),
    Column('fullname', String),
)

addresses = Table('addresses', metadata,
  Column('id', Integer, primary_key=True),
  Column('user_id', None, ForeignKey('users.id')),
  Column('email_address', String, nullable=False)
 )

In [4]:
metadata.create_all(engine)

2020-01-05 23:08:25,111 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-01-05 23:08:25,114 INFO sqlalchemy.engine.base.Engine ()
2020-01-05 23:08:25,120 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-01-05 23:08:25,121 INFO sqlalchemy.engine.base.Engine ()
2020-01-05 23:08:25,126 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("users")
2020-01-05 23:08:25,132 INFO sqlalchemy.engine.base.Engine ()
2020-01-05 23:08:25,143 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("users")
2020-01-05 23:08:25,146 INFO sqlalchemy.engine.base.Engine ()
2020-01-05 23:08:25,150 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("addresses")
2020-01-05 23:08:25,155 INFO sqlalchemy.engine.base.Engine ()
2020-01-05 23:08:25,161 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("addresses")
2020-01-05 23:08:25,169 INFO sqlalchemy.engine.base.Engine ()
2020-01-05 23:08:2

In [6]:
ins = users.insert()
str(ins)

'INSERT INTO users (id, name, fullname) VALUES (:id, :name, :fullname)'

In [7]:
ins = users.insert().values(name='jack', fullname='Jack Jones')
str(ins)

'INSERT INTO users (name, fullname) VALUES (:name, :fullname)'

In [8]:
>>> ins.compile().params  

{'name': 'jack', 'fullname': 'Jack Jones'}

- (https://docs.sqlalchemy.org/en/13/core/tutorial.html#executing)
- (https://docs.sqlalchemy.org/en/13/core/tutorial.html#conjunctions)


In [9]:
>>> conn = engine.connect()
>>> conn

In [10]:
result = conn.execute(ins)

2020-01-06 10:31:39,799 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname) VALUES (?, ?)
2020-01-06 10:31:39,802 INFO sqlalchemy.engine.base.Engine ('jack', 'Jack Jones')
2020-01-06 10:31:39,807 INFO sqlalchemy.engine.base.Engine COMMIT


In [11]:
>>> ins.bind = engine
>>> str(ins)

'INSERT INTO users (name, fullname) VALUES (?, ?)'

In [12]:
result.inserted_primary_key

[1]

In [13]:
ins = users.insert()
conn.execute(ins, id=2, name='wendy', fullname='Wendy Williams')

2020-01-06 10:33:09,726 INFO sqlalchemy.engine.base.Engine INSERT INTO users (id, name, fullname) VALUES (?, ?, ?)
2020-01-06 10:33:09,729 INFO sqlalchemy.engine.base.Engine (2, 'wendy', 'Wendy Williams')
2020-01-06 10:33:09,733 INFO sqlalchemy.engine.base.Engine COMMIT


In [14]:
conn.execute(addresses.insert(), [
   {'user_id': 1, 'email_address' : 'jack@yahoo.com'},
   {'user_id': 1, 'email_address' : 'jack@msn.com'},
   {'user_id': 2, 'email_address' : 'www@www.org'},
   {'user_id': 2, 'email_address' : 'wendy@aol.com'},
])

2020-01-06 10:33:25,470 INFO sqlalchemy.engine.base.Engine INSERT INTO addresses (user_id, email_address) VALUES (?, ?)
2020-01-06 10:33:25,472 INFO sqlalchemy.engine.base.Engine ((1, 'jack@yahoo.com'), (1, 'jack@msn.com'), (2, 'www@www.org'), (2, 'wendy@aol.com'))
2020-01-06 10:33:25,476 INFO sqlalchemy.engine.base.Engine COMMIT


## Selecting


In [15]:
>>> from sqlalchemy.sql import select
>>> s = select([users])
>>> result = conn.execute(s)

2020-01-06 10:34:00,145 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users
2020-01-06 10:34:00,148 INFO sqlalchemy.engine.base.Engine ()


In [16]:
>>> for row in result:
...     print(row)

(1, 'jack', 'Jack Jones')
(2, 'wendy', 'Wendy Williams')


In [17]:
>>> result = conn.execute(s)
>>> row = result.fetchone()
>>> print("name:", row['name'], "; fullname:", row['fullname'])

2020-01-06 10:34:28,613 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users
2020-01-06 10:34:28,615 INFO sqlalchemy.engine.base.Engine ()
name: jack ; fullname: Jack Jones


In [18]:
>>> row = result.fetchone()
>>> print("name:", row[1], "; fullname:", row[2])

name: wendy ; fullname: Wendy Williams


In [19]:
>>> for row in conn.execute(s):
...     print("name:", row[users.c.name], "; fullname:", row[users.c.fullname])

2020-01-06 10:34:46,410 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users
2020-01-06 10:34:46,413 INFO sqlalchemy.engine.base.Engine ()
name: jack ; fullname: Jack Jones
name: wendy ; fullname: Wendy Williams


In [20]:
>>> result.close()

In [21]:
s = select([users.c.name, users.c.fullname])
result = conn.execute(s)
for row in result:
    print(row)

2020-01-06 10:35:21,904 INFO sqlalchemy.engine.base.Engine SELECT users.name, users.fullname 
FROM users
2020-01-06 10:35:21,907 INFO sqlalchemy.engine.base.Engine ()
('jack', 'Jack Jones')
('wendy', 'Wendy Williams')


In [22]:
>>> for row in conn.execute(select([users, addresses])):
...     print(row)

2020-01-06 10:35:31,925 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname, addresses.id, addresses.user_id, addresses.email_address 
FROM users, addresses
2020-01-06 10:35:31,929 INFO sqlalchemy.engine.base.Engine ()
(1, 'jack', 'Jack Jones', 1, 1, 'jack@yahoo.com')
(1, 'jack', 'Jack Jones', 2, 1, 'jack@msn.com')
(1, 'jack', 'Jack Jones', 3, 2, 'www@www.org')
(1, 'jack', 'Jack Jones', 4, 2, 'wendy@aol.com')
(2, 'wendy', 'Wendy Williams', 1, 1, 'jack@yahoo.com')
(2, 'wendy', 'Wendy Williams', 2, 1, 'jack@msn.com')
(2, 'wendy', 'Wendy Williams', 3, 2, 'www@www.org')
(2, 'wendy', 'Wendy Williams', 4, 2, 'wendy@aol.com')


In [23]:
>>> s = select([users, addresses]).where(users.c.id == addresses.c.user_id)
>>> for row in conn.execute(s):
...     print(row)

2020-01-06 10:35:46,045 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname, addresses.id, addresses.user_id, addresses.email_address 
FROM users, addresses 
WHERE users.id = addresses.user_id
2020-01-06 10:35:46,049 INFO sqlalchemy.engine.base.Engine ()
(1, 'jack', 'Jack Jones', 1, 1, 'jack@yahoo.com')
(1, 'jack', 'Jack Jones', 2, 1, 'jack@msn.com')
(2, 'wendy', 'Wendy Williams', 3, 2, 'www@www.org')
(2, 'wendy', 'Wendy Williams', 4, 2, 'wendy@aol.com')


In [24]:
str(users.c.id == addresses.c.user_id)

'users.id = addresses.user_id'

In [25]:
print(users.c.id == 7)

users.id = :id_1


In [26]:
(users.c.id == 7).compile().params

{'id_1': 7}

In [27]:
print(users.c.name == None)

users.name IS NULL


In [28]:
print('fred' > users.c.name)

users.name < :name_1


$ pip install mysqlclient

In [29]:
# If we add two integer columns together, we get an addition expression:
print(users.c.id + addresses.c.id)
# Interestingly, the type of the Column is important! If we use + with two string based columns (recall we put types like Integer and String on our Column objects at the beginning), we get something different:
print(users.c.name + users.c.fullname)
# Where || is the string concatenation operator used on most databases. 
# But not all of them. MySQL users, fear not:
print((users.c.name + users.c.fullname).
     compile(bind=create_engine('mysql://'))) 

users.id + addresses.id
users.name || users.fullname


ModuleNotFoundError: No module named 'MySQLdb'

In [30]:
# If you have come across an operator which really isn’t available, 
# you can always use the Operators.op() method; 
# this generates whatever operator you need:
print(users.c.name.op('tiddlywinks')('foo'))

users.name tiddlywinks :name_1


## Conjunctions
We’d like to show off some of our operators inside of select() constructs. But we need to lump them together a little more, so let’s first introduce some conjunctions. Conjunctions are those little words like AND and OR that put things together. We’ll also hit upon NOT. and_(), or_(), and not_() can work from the corresponding functions SQLAlchemy provides (notice we also throw in a like()):

In [31]:
from sqlalchemy.sql import and_, or_, not_
print(and_(
        users.c.name.like('j%'),
        users.c.id == addresses.c.user_id,
        or_(
             addresses.c.email_address == 'wendy@aol.com',
             addresses.c.email_address == 'jack@yahoo.com'
        ),
        not_(users.c.id > 5)
      )
 )

users.name LIKE :name_1 AND users.id = addresses.user_id AND (addresses.email_address = :email_address_1 OR addresses.email_address = :email_address_2) AND users.id <= :id_1


In [32]:
print(users.c.name.like('j%') & (users.c.id == addresses.c.user_id) &
    (
      (addresses.c.email_address == 'wendy@aol.com') | \
      (addresses.c.email_address == 'jack@yahoo.com')
    ) \
    & ~(users.c.id>5)
)

users.name LIKE :name_1 AND users.id = addresses.user_id AND (addresses.email_address = :email_address_1 OR addresses.email_address = :email_address_2) AND users.id <= :id_1


In [33]:
>>> s = select([(users.c.fullname +
...               ", " + addresses.c.email_address).
...                label('title')]).\
...        where(
...           and_(
...               users.c.id == addresses.c.user_id,
...               users.c.name.between('m', 'z'),
...               or_(
...                  addresses.c.email_address.like('%@aol.com'),
...                  addresses.c.email_address.like('%@msn.com')
...               )
...           )
...        )
>>> conn.execute(s).fetchall()

2020-01-06 10:58:17,045 INFO sqlalchemy.engine.base.Engine SELECT users.fullname || ? || addresses.email_address AS title 
FROM users, addresses 
WHERE users.id = addresses.user_id AND users.name BETWEEN ? AND ? AND (addresses.email_address LIKE ? OR addresses.email_address LIKE ?)
2020-01-06 10:58:17,048 INFO sqlalchemy.engine.base.Engine (', ', 'm', 'z', '%@aol.com', '%@msn.com')


[('Wendy Williams, wendy@aol.com',)]

In [35]:
# A shortcut to using and_() is to chain together multiple where() clauses. The above can also be written as:

s = select([(users.c.fullname +
              ", " + addresses.c.email_address).
               label('title')]).\
       where(users.c.id == addresses.c.user_id).\
       where(users.c.name.between('m', 'z')).\
       where(
              or_(
                 addresses.c.email_address.like('%@aol.com'),
                 addresses.c.email_address.like('%@msn.com')
              )
       )
conn.execute(s).fetchall()

2020-01-06 10:58:57,454 INFO sqlalchemy.engine.base.Engine SELECT users.fullname || ? || addresses.email_address AS title 
FROM users, addresses 
WHERE users.id = addresses.user_id AND users.name BETWEEN ? AND ? AND (addresses.email_address LIKE ? OR addresses.email_address LIKE ?)
2020-01-06 10:58:57,458 INFO sqlalchemy.engine.base.Engine (', ', 'm', 'z', '%@aol.com', '%@msn.com')


[('Wendy Williams, wendy@aol.com',)]

In [36]:
# Using Textual SQL

from sqlalchemy.sql import text
s = text(
    "SELECT users.fullname || ', ' || addresses.email_address AS title "
        "FROM users, addresses "
        "WHERE users.id = addresses.user_id "
        "AND users.name BETWEEN :x AND :y "
        "AND (addresses.email_address LIKE :e1 "
            "OR addresses.email_address LIKE :e2)")
conn.execute(s, x='m', y='z', e1='%@aol.com', e2='%@msn.com').fetchall()

2020-01-06 11:27:36,468 INFO sqlalchemy.engine.base.Engine SELECT users.fullname || ', ' || addresses.email_address AS title FROM users, addresses WHERE users.id = addresses.user_id AND users.name BETWEEN ? AND ? AND (addresses.email_address LIKE ? OR addresses.email_address LIKE ?)
2020-01-06 11:27:36,472 INFO sqlalchemy.engine.base.Engine ('m', 'z', '%@aol.com', '%@msn.com')


[('Wendy Williams, wendy@aol.com',)]

In [37]:
# Specifying Bound Parameter Behaviors
stmt = text("SELECT * FROM users WHERE users.name BETWEEN :x AND :y")
stmt = stmt.bindparams(x="m", y="z")